In [1]:
import sys
print(sys.executable)


C:\Users\Latitude\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe


In [3]:
!pip install kaggle pandas zipfile36


In [ ]:
import pandas as pd

df = pd.read_csv("Breast_Cancer.csv")













print(df.head())
print(df.columns)


   Age   Race Marital Status T Stage  N Stage 6th Stage  \
0   68  White        Married       T1      N1       IIA   
1   50  White        Married       T2      N2      IIIA   
2   58  White       Divorced       T3      N3      IIIC   
3   58  White        Married       T1      N1       IIA   
4   47  White        Married       T2      N1       IIB   

               differentiate Grade   A Stage  Tumor Size Estrogen Status  \
0      Poorly differentiated     3  Regional           4        Positive   
1  Moderately differentiated     2  Regional          35        Positive   
2  Moderately differentiated     2  Regional          63        Positive   
3      Poorly differentiated     3  Regional          18        Positive   
4      Poorly differentiated     3  Regional          41        Positive   

  Progesterone Status  Regional Node Examined  Reginol Node Positive  \
0            Positive                      24                      1   
1            Positive                      1

In [7]:
print(df.columns.tolist())


['Age', 'T Stage ', 'N Stage', '6th Stage', 'differentiate', 'Grade', 'A Stage', 'Tumor Size', 'Estrogen Status', 'Progesterone Status', 'Regional Node Examined', 'Reginol Node Positive', 'Survival Months', 'Status']


In [9]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

import joblib

# 🔹 Step 1: Load text dataset
df = pd.read_csv("Breast_Cancer.csv")

# 🔹 Step 2: Encode categorical features
le_dict = {}  # store encoders for each column

for col in df.columns:
    if df[col].dtype == "object" and col != "Status":  # don't encode target
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str))
        le_dict[col] = le  # store each label encoder for decoding later

# 🔹 Step 3: Define features (X) and target (y)
X_text = df.drop("Status", axis=1).values
y_text = df["Status"].values

print("✅ X_text shape:", X_text.shape)
print("✅ y_text shape:", y_text.shape)

# 🔹 Step 4: Encode target labels
target_encoder = LabelEncoder()
y_text_encoded = target_encoder.fit_transform(y_text)

# 🔹 Step 5: Split the data
X_train_text, X_test_text, y_train_text, y_test_text = train_test_split(
    X_text, y_text_encoded, test_size=0.2, random_state=42
)

# 🔹 Step 6: Balance dataset using SMOTE
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train_text, y_train_text)

# 🔹 Step 7: Train RandomForest model
rf_text = RandomForestClassifier(n_estimators=200, random_state=42)
rf_text.fit(X_train_res, y_train_res)

# 🔹 Step 8: Evaluate model
y_pred_text = rf_text.predict(X_test_text)
y_pred_decoded = target_encoder.inverse_transform(y_pred_text)
y_test_decoded = target_encoder.inverse_transform(y_test_text)

print("📊 Accuracy:", accuracy_score(y_test_decoded, y_pred_decoded))
print(classification_report(y_test_decoded, y_pred_decoded))


# 🔹 Step 10: Save trained model and encoders for app.py
joblib.dump(rf_text, "text_model.pkl")
joblib.dump(target_encoder, "text_target_encoder.pkl")
joblib.dump(le_dict, "text_feature_encoders.pkl")

print("✅ Model and encoders saved successfully!")


✅ X_text shape: (4024, 15)
✅ y_text shape: (4024,)
📊 Accuracy: 0.8956521739130435
              precision    recall  f1-score   support

       Alive       0.93      0.95      0.94       685
        Dead       0.67      0.60      0.63       120

    accuracy                           0.90       805
   macro avg       0.80      0.77      0.79       805
weighted avg       0.89      0.90      0.89       805

✅ Model and encoders saved successfully!
